# Agents

In [24]:
from pydantic import BaseModel, Field
import json

class PasswordLookupRequest(BaseModel):
    """Request for the password_lookup Tool"""
    password_index: int|str = Field(..., title="password_index", description="Index of the Password to check")


In [54]:
def password_lookup(request: PasswordLookupRequest):
    """Returns the password at the given index"""
    password = [
        'At', 'Woven', 'by', 'Toyota','we'
    ]
    return password[int(request.password_index)]

In [55]:
type(password_lookup.__annotations__["request"])

pydantic._internal._model_construction.ModelMetaclass

In [56]:
from pydantic import TypeAdapter

TypeAdapter(password_lookup.__annotations__["request"]).validate_json('{"password_index": 2}')

PasswordLookupRequest(password_index=2)

In [57]:
password_lookup.__doc__

'Returns the password at the given index'

In [58]:
json.dumps([1,2,3])

'[1, 2, 3]'

In [59]:
tool_prompts = "Tools are available to help you. If you wish to use one of these, output JSON in the form:\n['tool_0_name': tool_0_kwargs, 'tool_1_name': tool_1_kwargs, ...]\n Where tool_i is the name of the tool and tool_i_kwargs is a dictionary of the keyword arguments for the tool. The following tools are available to you:\n"
for tool in [password_lookup]:
    tool_prompts += f"\n{tool.__name__}: {tool.__doc__}"

    schema = password_lookup.__annotations__["request"].model_json_schema()
    tool_prompts += f"\nThe tool kwargs will be used for {schema['description']}, and expect the following fields:"
    tool_prompts += '\n\n{'
    for field in schema['properties'].keys():
        tool_prompts += f"\n  {field}: {schema['properties'][field]['type']}  # {schema['properties'][field]['description']}"
    tool_prompts += '\n}\n\n'

tool_prompts += "If you request to use one or more tools, then the output will be fed back to you "

In [60]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [61]:
schema

{'description': 'Request format for the password_lookup Tool',
 'properties': {'password_index': {'description': 'Index of the Password to check',
   'title': 'password_index',
   'type': 'integer'}},
 'required': ['password_index'],
 'title': 'PasswordLookupRequest',
 'type': 'object'}

In [62]:
tools = []
for tool in [password_lookup]:
    schema = tool.__annotations__["request"].model_json_schema()
    tools.append({
        "type": "function",
        "function": {
            "name": tool.__name__,
            "description": tool.__doc__,
            "parameters": {"additionalProperties": False, **schema},
            "strict": True
        }
    })

In [63]:
print(tools)

[{'type': 'function', 'function': {'name': 'password_lookup', 'description': 'Returns the password at the given index', 'parameters': {'additionalProperties': False, 'description': 'Request format for the password_lookup Tool', 'properties': {'password_index': {'description': 'Index of the Password to check', 'title': 'password_index', 'type': 'integer'}}, 'required': ['password_index'], 'title': 'PasswordLookupRequest', 'type': 'object'}, 'strict': True}}]


In [123]:
from openai import OpenAI
from typing import Callable

class LLMWithTools:
    def __init__(self, tools: list[Callable]):
        self.client = OpenAI()
        self.tools = []
        self.tool_names = {tool.__name__: tool for tool in tools}
        for tool in tools:
            schema = tool.__annotations__["request"].model_json_schema()
            self.tools.append({
                "type": "function",
                "function": {
                    "name": tool.__name__,
                    "description": tool.__doc__,
                    "parameters": {"additionalProperties": False, **schema},
                    "strict": False
                }
            })
    
    def __call__(self, text: str) -> str:
        messages = [
            {"role": "user", "content": text}
        ]

        def call():
            return self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=self.tools
            )
        
        response = call()
        while response.choices[0].message.tool_calls is not None and len(response.choices[0].message.tool_calls) > 0:
            messages.append(response.choices[0].message)
            for tool_call in response.choices[0].message.tool_calls:
                if tool_call.function.name in self.tool_names:
                    f = self.tool_names[tool_call.function.name]
                    t = f.__annotations__["request"]
                    print(f"Calling {f.__name__} with {t}")
                    tool_response = self.tool_names[tool_call.function.name](TypeAdapter(t).validate_json(tool_call.function.arguments))
                    messages.append({"role": "tool", "content": tool_response, "tool_call_id": tool_call.id,})
            response = call()

        return response.choices[0].message.content

In [124]:
llm = LLMWithTools([password_lookup])

In [105]:
llm("Hello, I would like to use the password_lookup tool. How do I use it?")

'The `password_lookup` tool allows you to retrieve a password using its index. To use it, you need to provide a single parameter: `password_index`, which is an integer representing the index of the password you want to look up.\n\nHere\'s an example format of how you would request a password lookup:\n\n```json\n{\n  "password_index": 0\n}\n```\n\nIn this example, you would replace `0` with the index of the password you desire. If you let me know which index you want to look up, I can assist you in making the request!'

In [126]:
llm("What is the password at index 2?")

Calling password_lookup with <class '__main__.PasswordLookupRequest'>


'The password at index 2 is "by".'